Импорт библиотек

In [264]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv

Прочитаем значения переменных окружения

In [266]:
load_dotenv() # загружаются переменные из файла

DATE_BEGIN = os.getenv('DATE_BEGIN')
DATE_END   = os.getenv('DATE_END')
API_URL    = os.getenv('API_URL')

Запрос данных по визитам

In [268]:
l_requests_visits = requests.get(f'{API_URL}/visits', params={'begin': DATE_BEGIN, 'end': DATE_END})
l_df_api_visits = pd.DataFrame.from_dict(l_requests_visits.json())

Запрос данных по регистрациям

In [270]:
l_requests_regs = requests.get(f'{API_URL}/registrations', params={'begin': DATE_BEGIN, 'end': DATE_END})
l_df_api_regs = pd.DataFrame.from_dict(l_requests_regs.json())

Отфильтруем ботов

In [272]:
l_df_api_visits['bot'] = l_df_api_visits['user_agent'].str.contains('bot')
l_df_api_visits = l_df_api_visits[~l_df_api_visits['bot']]

Рассчитаем поле с датой в формате Unix

In [274]:
l_df_api_visits['date'] = l_df_api_visits['datetime'].str.slice(stop=10)
l_df_api_visits['date_group'] = pd.to_datetime(l_df_api_visits['date']).astype('int64').astype(str).str.slice(stop=13)

Удаление дубликатов и сортировка

In [276]:
l_df_api_visits = l_df_api_visits[['date_group', 'platform', 'visit_id']]
l_df_api_visits.drop_duplicates(inplace=True)
l_df_api_visits.sort_values(['date_group','platform'], ascending=[True, True], inplace=True)

Сагрегируем таблицу визитов по датам посещений и платформам

In [278]:
l_df_api_visits_agg = l_df_api_visits.groupby(['date_group', 'platform']).agg('count')
l_df_api_visits_agg.reset_index(inplace=True)
l_df_api_visits_agg.rename(columns={'visit_id': 'visits'}, inplace=True)

Рассчитаем поле с датой в формате Unix

In [280]:
l_df_api_regs['date'] = l_df_api_regs['datetime'].str.slice(stop=10)
l_df_api_regs['date_group'] = pd.to_datetime(l_df_api_regs['date']).astype('int64').astype(str).str.slice(stop=13)

Удалим дубликаты

In [282]:
l_df_api_regs = l_df_api_regs[['date_group', 'platform', 'user_id']]
l_df_api_regs.drop_duplicates(inplace=True)

Сагрегируем таблицу регистраций по датам посещений и платформам

In [284]:
l_df_api_regs_agg = l_df_api_regs.groupby(['date_group', 'platform']).agg('count')
l_df_api_regs_agg.reset_index(inplace=True)
l_df_api_regs_agg.rename(columns={'user_id': 'registrations'}, inplace=True)

Объединим таблицы посещений и регистраций. Рассчитаем конверсию.

In [290]:
l_df_conversion = pd.merge(l_df_api_visits_agg, l_df_api_regs_agg, on=['date_group', 'platform'], how='outer')
l_df_conversion['conversion'] = round(l_df_conversion['registrations'] / l_df_conversion['visits'], 10)
l_df_conversion.sort_values(['date_group', 'platform'], ascending=[True, True], inplace=True)
l_df_conversion.reset_index(drop=True, inplace=True)
l_df_conversion.to_json('./conversion.json')

Прочитаем данные о рекламных кампаниях

In [294]:
l_df_ads = pd.read_csv('./ads.csv')

Рассчитаем поле с датой и агрегируем таблицу

In [296]:
l_df_ads['date'] = l_df_ads['date'].str.slice(stop=10)
l_df_ads = l_df_ads[l_df_ads['date'] < DATE_END]
l_df_ads['date_group'] = pd.to_datetime(l_df_ads['date']).astype('int64').astype(str).str.slice(stop=13)
l_df_ads_agg_2 = l_df_ads.groupby(['date_group', 'utm_campaign']).agg({'cost': 'sum'})
l_df_ads_agg_2.reset_index(inplace=True)

Ещё больше агрегируем таблицы визитов и регистраций (поле platform больше не нужно)

In [298]:
l_df_api_visits_agg_2 = l_df_api_visits_agg.groupby('date_group').agg({'visits': 'sum'})
l_df_api_regs_agg_2 = l_df_api_regs_agg.groupby('date_group').agg({'registrations': 'sum'})

Рассчитываем таблицу с зарплатами на рекламу

In [300]:
l_df_merge_ads_1 = pd.merge(l_df_api_visits_agg_2, l_df_api_regs_agg_2, on=['date_group'], how='outer')
l_df_merge_ads_1.reset_index(inplace=True)
l_df_merge_ads_2 = pd.merge(l_df_merge_ads_1, l_df_ads_agg_2, on=['date_group'], how='outer')
l_df_ads_2 = l_df_merge_ads_2[['date_group', 'visits', 'registrations', 'cost', 'utm_campaign']]
l_df_ads_2.fillna({'cost': 0, 'utm_campaign': 'None'}, inplace=True)
l_df_ads_2.sort_values(['date_group'], ascending=[True], inplace=True)
l_df_ads_2.reset_index(drop=True, inplace=True)
l_df_ads_2.to_json('./ads.json')